In [1]:
import polars as pl
import numpy as np
import data_actions.data_processing.utils as utils

In [2]:
data = utils.load_data("data/original_data")
data_cart = data["events"]["add_to_cart"]
data_cart.collect()

/home/adam/python-projects/recsys25/data_actions/data_processing/utils.py:67: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  if "timestamp" in df.columns:


client_id,timestamp,sku
i64,datetime[μs],i64
17649961,2022-08-11 12:22:55,1398840
315805,2022-09-11 05:34:10,1434568
315805,2022-10-06 04:58:15,998226
16696114,2022-07-11 16:27:00,81192
10238779,2022-10-05 14:36:55,1475246
…,…,…
16225940,2022-10-10 16:07:50,498350
3850845,2022-06-12 16:13:59,1461294
12296853,2022-07-29 17:53:45,431834


In [3]:
from collections import defaultdict

k = 100

# Sort data_cart by timestamp
data_cart_chronological = data_cart.sort("timestamp").collect()

# Mapa klient -> lista sku
client_skus = defaultdict(list)

for row in data_cart_chronological.iter_rows(named=True):
    client_id = row["client_id"]
    sku = row["sku"]
    if len(client_skus[client_id]) < k:
        client_skus[client_id].append(sku)

# Zamień na listę słowników z kolumnami sku_0, sku_1, ..., sku_{k-1}
rows = []
for client_id, skus in client_skus.items():
    row = {"client_id": client_id}
    for i, sku in enumerate(skus):
        row[f"sku_{i}"] = sku
    rows.append(row)

# Utwórz DataFrame
V = pl.DataFrame(rows)


In [4]:
V

client_id,sku_0,sku_1,sku_2,sku_3,sku_4,sku_5,sku_6,sku_7,sku_8,sku_9,sku_10,sku_11,sku_12,sku_13,sku_14,sku_15,sku_16,sku_17,sku_18,sku_19,sku_20,sku_21,sku_22,sku_23,sku_24,sku_25,sku_26,sku_27,sku_28,sku_29,sku_30,sku_31,sku_32,sku_33,sku_34,sku_35,…,sku_63,sku_64,sku_65,sku_66,sku_67,sku_68,sku_69,sku_70,sku_71,sku_72,sku_73,sku_74,sku_75,sku_76,sku_77,sku_78,sku_79,sku_80,sku_81,sku_82,sku_83,sku_84,sku_85,sku_86,sku_87,sku_88,sku_89,sku_90,sku_91,sku_92,sku_93,sku_94,sku_95,sku_96,sku_97,sku_98,sku_99
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
21033801,176746,743987,449567,1221307,739980,147793,968890,282541,465709,449567,1389605,1046080,579613,579613,594628,147793,1482453,147793,1482453,1477516,147793,1482453,282541,859298,1362002,1094946,1370849,1452470,671008,1444488,594043,597851,824778,282541,450510,488348,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
22958544,1116672,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1183278,1311713,985371,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
3403615,32189,1388714,971353,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
18592960,359186,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
403369,378558,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
8380305,1466277,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
726161,806607,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,n

In [7]:
target = pl.read_parquet("data/original_data/target/train_target.parquet")

In [18]:
def create_inputs(target):
    """inputs creation

    Args:
        target (pl.DataFrame): train_target dataframe
    """
    target.sort("timestamp")
    inputs = defaultdict(list)
    for row in target.iter_rows(named=True):
        client_id = row["client_id"]
        sku = row["sku"]
        inputs[client_id].append(sku)

    for client in inputs:
        inputs[client].append(-1)


    clients, inputs = zip(*inputs.items())
    return clients, inputs

In [15]:
clients, inputs = create_inputs(target)